# Анализ тональности текста с помощью Python


Данная статья представляет собой незначительно сокращенный (и пока что частичный) перевод статьи [Кайла Стратиса](https://realpython.com/sentiment-analysis-python/).

---

[Анализ тональности](https://ru.wikipedia.org/wiki/%D0%90%D0%BD%D0%B0%D0%BB%D0%B8%D0%B7_%D1%82%D0%BE%D0%BD%D0%B0%D0%BB%D1%8C%D0%BD%D0%BE%D1%81%D1%82%D0%B8_%D1%82%D0%B5%D0%BA%D1%81%D1%82%D0%B0) (сентимент-анализ) – инструмент компьютерной лингвистики, оценивающий такую субъективную составляющую текста, как отношение пишущего. Часто это составляет трудность даже для опытных читателей — что уж говорить о программах. Однако эмоциональную окраску текста все-таки можно проанализировать с помощью инструментов экосистемы Python.

Зачем это может быть нужно? Существует множество применений для анализа тональности. Например, такие данные позволяют предсказать поведение биржевых трейдеров относительно конкретной компании по откликам в социальных сетях и другим отзывам.

В этом руководстве мы рассмотрим:
- как использовать методы [обработки естественного языка](https://ru.wikipedia.org/wiki/%D0%9E%D0%B1%D1%80%D0%B0%D0%B1%D0%BE%D1%82%D0%BA%D0%B0_%D0%B5%D1%81%D1%82%D0%B5%D1%81%D1%82%D0%B2%D0%B5%D0%BD%D0%BD%D0%BE%D0%B3%D0%BE_%D1%8F%D0%B7%D1%8B%D0%BA%D0%B0) (англ. natural language processing, NLP);
- как использовать машинное обучение для определения тональности текста;
- как использовать библиотеку spaCy для создания классификатора анализа настроений.

Это руководство предназначено для начинающих практиков машинного обучения.


# Предварительная обработка и очистка текстовых данных

Любой рабочий процесс анализа данных начинается с их загрузки. Но после того, как данные будут загружены, мы должны будем пропустить их через своеобразный конвейер (pipeline) обработки естественного языка.

В частности, нам потребуется сделать следующие шаги:
- **токенизировать текст** – разбить текст на предложения, слова и другие единицы;
- **удалить стоп-слова** – «если», «но», «или» и т. д.;
- **привести слова к нормальной форме**;
- **привести текст к векторной форме** – сделать числовое представление текста для его дальнейшей обработки классификатором.

Все эти шаги служат для уменьшения шума, присущего любому обычному тексту, и повышения точности результатов классификатора. Эти задачи решают множество отличных библиотек – [NLTK](https://www.nltk.org/), [TextBlob](https://textblob.readthedocs.io/en/dev/index.html) и [spaCy](https://spacy.io/) – последнюю мы и будем применять в этом руководстве мы будем использовать spaCy.

Прежде чем идти дальше, убедитесь, что у вас установлен spaCy и модель для английского язка (расскомментриуйте две следующие строки, чтобы установить библиотеку и модуль):

In [ ]:
#!pip install spacy
#!python -m spacy download en_core_web_sm

---
**Примечание переводчика**. Об использовании spaCy для русского языка подробно написано в README-файле репозитория [spaCy](https://github.com/buriy/spacy-ru).

---

# Токенизация

Токенизация – это процесс разбиения текста на более мелкие части. Библиотека spaCy поставляется вместе с конвейером, начинающем обработку текста с токенизации. Можно выполнять токенизацию предложений и слов. В этом руководстве мы разделим текст на отдельные слова. Загрузим текст примера и проведем разбиение:

In [3]:
import spacy
nlp = spacy.load("en_core_web_sm")

text = """
Dave watched as the forest burned up on the hill,
only a few miles from his house. The car had
been hastily packed and Marta was inside trying to round
up the last of the pets. "Where could she be?" he wondered
as he continued to wait for Marta to appear with the pets.
"""

doc = nlp(text)
token_list = [token for token in doc]

print(token_list)

[
, Dave, watched, as, the, forest, burned, up, on, the, hill, ,, 
, only, a, few, miles, from, his, house, ., The, car, had, 
, been, hastily, packed, and, Marta, was, inside, trying, to, round, 
, up, the, last, of, the, pets, ., ", Where, could, she, be, ?, ", he, wondered, 
, as, he, continued, to, wait, for, Marta, to, appear, with, the, pets, ., 
]


Вы могли заметить, что захваченные токены включают знаки препинания и другие строки, не относящиеся к словам. Это нормальное поведение в случае использований конвейера по умолчанию.

# Удаление стоп-слов

Стоп-слова – это слова, которые могут иметь важное значение в человеческом общении, но не имеют большого значения для машин. spaCy поставляется со списком стоп-слов по умолчанию (его можно настроить). Проведем фильтрацию полученного списка:

In [4]:
filtered_tokens = [token for token in doc if not token.is_stop]
print(filtered_tokens)

[
, Dave, watched, forest, burned, hill, ,, 
, miles, house, ., car, 
, hastily, packed, Marta, inside, trying, round, 
, pets, ., ", ?, ", wondered, 
, continued, wait, Marta, appear, pets, ., 
]


Одной строкой Python-кода мы отфильтровали стоп-слова из токенизированного текста с помощью атрибута токена `.is_stop`. После удаления стоп-слов список токенов стал короче, исчезли местоимения и служебные слова: артикли, союзы, предлоги и послелоги.

# Приведение к нормальной форме
Процесс **нормализации** несколько сложнее токенизации. Все формы слова мы должны привести к единому представлению. Например, `watched`, `watched` и `watches` после нормализации превращаются в `watch`. Есть два основных подхода к нормализации:
1. стемминг;
2. лемматизация.

В случае **стемминга** выделяется основа слова, дополнив которую можно получить слова-потомки. Такой метод сработает на приведенном примере. Однако это слишком простой подход – стемминг просто обрезает строку, отбрасывая окончание. Стемминг не обнаружит связь между `feel` и `felt`.

**Лемматизация** стремится решить указанную проблему, используя структуру данных, в которой все формы слова связаны с его простейшей формой – леммой. Лемматизация обычно приносит больше пользы, чем стемминг, и потому является единственной стратегия нормализации, предлагаемая spaCy.

В рамках NLP-конвейера лемметизация происходит автоматически (вместе с рядом других действий). Лемма для каждого токена хранится в атрибуте `.lemma_`:

In [7]:
lemmas = [
    f"Token: {token}, lemma: {token.lemma_}"
    for token in filtered_tokens
]

print(lemmas)

['Token: \n, lemma: \n', 'Token: Dave, lemma: Dave', 'Token: watched, lemma: watch', 'Token: forest, lemma: forest', 'Token: burned, lemma: burn', 'Token: hill, lemma: hill', 'Token: ,, lemma: ,', 'Token: \n, lemma: \n', 'Token: miles, lemma: mile', 'Token: house, lemma: house', 'Token: ., lemma: .', 'Token: car, lemma: car', 'Token: \n, lemma: \n', 'Token: hastily, lemma: hastily', 'Token: packed, lemma: pack', 'Token: Marta, lemma: Marta', 'Token: inside, lemma: inside', 'Token: trying, lemma: try', 'Token: round, lemma: round', 'Token: \n, lemma: \n', 'Token: pets, lemma: pet', 'Token: ., lemma: .', 'Token: ", lemma: "', 'Token: ?, lemma: ?', 'Token: ", lemma: "', 'Token: wondered, lemma: wonder', 'Token: \n, lemma: \n', 'Token: continued, lemma: continue', 'Token: wait, lemma: wait', 'Token: Marta, lemma: Marta', 'Token: appear, lemma: appear', 'Token: pets, lemma: pet', 'Token: ., lemma: .', 'Token: \n, lemma: \n']


---

**Примечание**. Обратите внимание на символ подчеркивания в атрибуте `.lemma_`. Это не опечатка, а результат соглашения по именованию в spaCy   версий атрибутов, которые могут быть прочитатаны человеком.

---

Следующим шагом является представление каждого токена способом, понятным машине. Этот процесс называется векторизацией.


# Векторизация текста

Векторизация – преобразование токена в вектор (числовой массив), который представляет его свойства и в контексте задачи является уникальным для каждого токена. Векторные представления токенов используются в поиске сходства слов, классификации текстов и т. д.

В spaCy токены векторизуются в виде плотный массивов, в которых определены значения для каждой позиции массива. Это отличает используемый подход от ранних методов, в которых для тех же целей применялись разреженные массивы, а большинство позиций были заполнены нулями.

Как и другие шаги, векторизация выполняется автоматически посредством вызова `nlp()`. Поскольку у нас уже есть список объектов токенов, мы можем получить векторное представление одного из токенов следующим образом:

In [8]:
filtered_tokens[1].vector

array([ 1.8371646 ,  1.4529226 , -1.6147211 ,  0.678362  , -0.6594443 ,
        1.6417935 ,  0.5796405 ,  2.3021278 , -0.13260496,  0.5750932 ,
        1.5654886 , -0.6938864 , -0.59607106, -1.5377437 ,  1.9425622 ,
       -2.4552505 ,  1.2321601 ,  1.0434952 , -1.5102385 , -0.5787632 ,
        0.12055647,  3.6501784 ,  2.6160972 , -0.5710199 , -1.5221789 ,
        0.00629176,  0.22760668, -1.922073  , -1.6252862 , -4.226225  ,
       -3.495663  , -3.312053  ,  0.81387717, -0.00677544, -0.11603224,
        1.4620426 ,  3.0751472 ,  0.35958546, -0.22527039, -2.743926  ,
        1.269633  ,  4.606786  ,  0.34034157, -2.1272311 ,  1.2619178 ,
       -4.209798  ,  5.452852  ,  1.6940253 , -2.5972986 ,  0.95049495,
       -1.910578  , -2.374927  , -1.4227567 , -2.2528825 , -1.799806  ,
        1.607501  ,  2.9914255 ,  2.8065152 , -1.2510269 , -0.54964066,
       -0.49980402, -1.3882618 , -0.470479  , -2.9670253 ,  1.7884955 ,
        4.5282774 , -1.2602427 , -0.14885521,  1.0419178 , -0.08

Здесь мы используем атрибут `.vector` для второго токена в списке `filter_tokens`. В этом наборе это слово `Dave`.

---

**Примечание**. Если для атрибута `.vector` вы получите другой результат, не беспокойтесь. Это может быть связано с тем, что используется другая версия модели `en_core_web_sm` или самой библиотеки `spaCy`.

---

Теперь, когда вы узнали о некоторых типичных этапах предварительной обработки текста в spaCy, узнаем, как классифицировать текст.


# Использование классификаторов машинного обучения для прогнозирования настроений